# Spatstat playground

Let's play with the [`spatstat`](https://spatstat.org/) (Spatial Statistics) R package from Python! 

To do this, one can use the Python package [`rpy2`](https://rpy2.github.io/doc/v3.4.x/html/overview.html#installation) to ensure the interoperability with [`R`](https://www.r-project.org/).
In other words, `rpy2` allows us to call `R` from `Python`.

In [ ]:
%config InlineBackend.figure_format='retina'

%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from spatstat_interface.interface import SpatstatInterface

spatstat = SpatstatInterface(update=False)

In [ ]:
spatstat.core

In [ ]:
spatstat.import_package("core", "geom", update=False)

In [ ]:
spatstat.core